# Demo for uproot

In [133]:
import numpy as np
import uproot
import awkward as ak

num_events = 1000 #must be match to the existed root files name
data_path = "/Users/yianchen/CMS_Open_Data_Workspace/CMSSW_7_6_7/src/QCD_Jet_Fatjet/Analyzer/root_files/"
channels = [
    "ZprimeToZhToZinvhbb",
    "ZprimeToZhToZlephbb",
    "QCD_HT1500to2000",
    "QCD_HT2000toInf",
]

def get_events(idx):
    print(channels[idx])
    channel = channels[idx]
    root_file = f"{data_path}{channel}_{num_events}.root"
    events = uproot.open(root_file+f":jets/Events")
    return events

### Cut by Pt

In [138]:
events = get_events(1)
cut_events = events.arrays(
    expressions = ["fatjet_pt", "fatjet_eta", "fatjet_phi"],
    cut = "(fatjet_pt >= 800) & (fatjet_pt <= 1200)"
)

max_pt_idx = ak.firsts(ak.argsort(cut_events["fatjet_pt"], ascending=False))
max_pt_idx = max_pt_idx[ak.is_none(max_pt_idx) == False]

cut_events = cut_events[max_pt_idx]
cut_events

ZprimeToZhToZlephbb


<Array [{fatjet_pt: [1.17e+03, ... 1.18]}] type='203 * ?{"fatjet_pt": var * floa...'>